In [1]:
import gpflow
import numpy as np

import matplotlib.pyplot as plt
import tensorflow as tf

import gpflow
from gpflow.utilities import ops, print_summary
from gpflow.config import set_default_float, default_float, set_default_summary_fmt
from gpflow.ci_utils import ci_niter

set_default_float(np.float64)
set_default_summary_fmt("notebook")

%matplotlib inline

In [3]:
(xtrain, ytrain), (xtest, ytest) = tf.keras.datasets.mnist.load_data()

In [11]:
Y = tf.reshape(tf.convert_to_tensor(xtrain, dtype=default_float()), [xtrain.shape[0], -1])

In [14]:
labels = tf.convert_to_tensor(ytrain)

In [17]:
latent_dim = 2  # number of latent dimensions
num_inducing = 20  # number of inducing pts
num_data = Y.shape[0]  # number of data points

In [18]:
x_mean_init = tf.convert_to_tensor(ops.pca_reduce(Y, latent_dim), dtype=default_float())
x_var_init = tf.convert_to_tensor(np.ones((num_data, latent_dim)), dtype=default_float())

In [20]:
inducing_variable = tf.convert_to_tensor(np.random.permutation(x_mean_init.numpy())[:num_inducing], dtype=default_float())

In [34]:
lengthscale = tf.convert_to_tensor([10.] * latent_dim, dtype=default_float())
kernel = gpflow.kernels.RBF(lengthscale=lengthscale)

In [35]:
print_summary(kernel)

name,class,transform,prior,trainable,shape,dtype,value
SquaredExponential.variance,Parameter,Softplus,,True,(),float64,1.0
SquaredExponential.lengthscale,Parameter,Softplus,,True,"(2,)",float64,[10. 10.]


In [36]:
gplvm = gpflow.models.BayesianGPLVM(Y,
            x_data_mean=x_mean_init,
            x_data_var=x_var_init,
            kernel=kernel,
            inducing_variable=inducing_variable)
# Instead of passing an inducing_variable directly, we can also set the num_inducing_variables argument to an integer, which will randomly pick from the data.

In [37]:
gplvm.likelihood.variance.assign(0.01)

In [38]:
print_summary(gplvm)

name,class,transform,prior,trainable,shape,dtype,value
BayesianGPLVM.kernel.variance,Parameter,Softplus,,True,(),float64,1.0
BayesianGPLVM.kernel.lengthscale,Parameter,Softplus,,True,"(2,)",float64,[10. 10.]
BayesianGPLVM.likelihood.variance,Parameter,Softplus + AffineScalar,,True,(),float64,0.009999999776482686
BayesianGPLVM.x_data_mean,Parameter,,,True,"(60000, 2)",float64,"[[-312.67426202, -123.93258866..."
BayesianGPLVM.x_data_var,Parameter,Softplus,,True,"(60000, 2)",float64,"[[1., 1...."
BayesianGPLVM.inducing_variable.Z,Parameter,,,True,"(20, 2)",float64,"[[-308.88899704, -1156.12183858..."


In [39]:
num_steps = 10000

@tf.function(autograph=False)
def optimization_step():
    return - gplvm.log_marginal_likelihood()

optimizer = tf.keras.optimizers.Adam()
for i in range(num_steps):
    optimizer.minimize(optimization_step, var_list=gplvm.trainable_variables)
    if i % 500 == 0:
        print('Negative log marginal likelihood at step %s: %s' % (i, -gplvm.log_marginal_likelihood().numpy()))

Negative log marginal likelihood at step 0: 17054350315386.973


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\seb\.conda\envs\gpflow\lib\site-packages\IPython\core\interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-39-c59e97494e09>", line 9, in <module>
    optimizer.minimize(optimization_step, var_list=gplvm.trainable_variables)
  File "C:\Users\seb\.conda\envs\gpflow\lib\site-packages\tensorflow_core\python\keras\optimizer_v2\optimizer_v2.py", line 317, in minimize
    loss, var_list=var_list, grad_loss=grad_loss)
  File "C:\Users\seb\.conda\envs\gpflow\lib\site-packages\tensorflow_core\python\keras\optimizer_v2\optimizer_v2.py", line 356, in _compute_gradients
    grads = tape.gradient(loss_value, var_list, grad_loss)
  File "C:\Users\seb\.conda\envs\gpflow\lib\site-packages\tensorflow_core\python\eager\backprop.py", line 1014, in gradient
    unconnected_gradients=unconnected_gradients)
  File "C:\Users\seb\.conda\envs\gpflow\lib\site-packages\tensorflow_core\python\ea

KeyboardInterrupt: 